# EX7-STRUCT: SparkSQL aggregates, execution plan, and others

Your assignment: complete the `TODO`'s and include also the **output of each cell**.

#### You may need to read the [Dataframe API Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html) to complete this lab.

### Step 1: Download Bike Trip Data (GDrive)

In [1]:
import os
import gdown

# stations data
if not os.path.isfile('data/station-data.json'):
    id = "134kLURYaTZuj6SWrg1XvkQKQ30bf1X7I"
    gdown.download(id=id, output="data/")
    print("Stations data downloaded")
else:
    print("Stations data already downloaded")

# trips data
if not os.path.isfile('data/trip-data.json'):
    id = "1pX3WHi3R2n52zyo6swXxlOtVTgt_hOQG"
    gdown.download(id=id, output="data/")
    print("Trips data downloaded")
else:
    print("Trips data already downloaded")

Stations data already downloaded
Trips data already downloaded


### Step 2: Start Spark Session

In [2]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

# local mode
spark = SparkSession.builder \
            .appName("Spark SQL basic example") \
            .master("local[*]") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

# cluster mode
#spark = SparkSession.builder \
#            .appName("Spark SQL basic example") \
#            .master("spark://spark:7077") \
#	    	.config("spark.some.config.option", "some-value") \
#	    	.getOrCreate()

SparkContext not defined


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 00:56:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/08 00:56:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Step 3: Read JSON files as dataframes

In [3]:
stations_df = spark.read.json('data/station-data.json')
stations_df.show()

+---------+------------+------------+---------+-----------+--------------------+----------+
|dockcount|installation|    landmark|      lat|       long|                name|station_id|
+---------+------------+------------+---------+-----------+--------------------+----------+
|       27|  2013-08-06|    San Jose|37.329732|-121.901782|San Jose Diridon ...|         2|
|       15|  2013-08-05|    San Jose|37.330698|-121.888979|San Jose Civic Ce...|         3|
|       11|  2013-08-06|    San Jose|37.333988|-121.894902|Santa Clara at Al...|         4|
|       19|  2013-08-05|    San Jose|37.331415|  -121.8932|    Adobe on Almaden|         5|
|       15|  2013-08-07|    San Jose|37.336721|-121.894074|    San Pedro Square|         6|
|       15|  2013-08-07|    San Jose|37.333798|-121.886943|Paseo de San Antonio|         7|
|       15|  2013-08-05|    San Jose|37.330165|-121.885831| San Salvador at 1st|         8|
|       15|  2013-08-05|    San Jose|37.348742|-121.894715|           Japantown|

In [4]:
trips_df = spark.read.json('data/trip-data.json').repartition(9)
trips_df.show()

+-------+--------+--------------------+--------------------+------------+--------------------+--------------------+--------------+---------------+-------+--------+
|bike_id|duration|            end_date|         end_station|end_terminal|          start_date|       start_station|start_terminal|subscriber_type|trip_id|zip_code|
+-------+--------+--------------------+--------------------+------------+--------------------+--------------------+--------------+---------------+-------+--------+
|    553|     698|2015-08-26T00:09:...|     Spear at Folsom|          49|2015-08-26T00:08:...|San Francisco Cal...|            69|     Subscriber| 905821|   94040|
|    292|     628|2015-08-20T00:09:...|  Powell Street BART|          39|2015-08-20T00:09:...|     Townsend at 7th|            65|     Subscriber| 898176|   94107|
|    506|     503|2015-08-28T00:08:...|       2nd at Folsom|          62|2015-08-28T00:08:...|Harry Bridges Pla...|            50|     Subscriber| 909314|   94558|
|    520|     44

### Step 4: Follow the [groupBy tutorial on SparkByExample](https://sparkbyexamples.com/pyspark/pyspark-groupby-explained-with-example/) `#TODO`
1. Reproduce in this notebook each example on the tutorial page, **however, using the stations and trips dataframes instead**
2. The objective is for you to take practice on each operator and hence, you may choose the specifics -- column, or columns, queries, etc. 

In [5]:
# Example 1: Group by a single column and count
stations_count = trips_df.groupBy("start_station").count()
stations_count.show()

# Example 2: Group by multiple columns
station_zip_count = trips_df.groupBy("start_station", "zip_code").count()
station_zip_count.show()

# Example 3: Group by a column and aggregate using sum
total_duration = trips_df.groupBy("start_station").sum("duration")
total_duration.show()

# Example 4: Group by a column and aggregate using avg
average_duration = trips_df.groupBy("start_station").avg("duration")
average_duration.show()

# Example 5: Group by a column and aggregate using min
min_duration = trips_df.groupBy("start_station").min("duration")
min_duration.show()

# Example 6: Group by a column and aggregate using max
max_duration = trips_df.groupBy("start_station").max("duration")
max_duration.show()

# Example 7: Group by a column and use multiple aggregations
station_duration_stats = trips_df.groupBy("start_station").agg(
    {"duration": "sum", "duration": "avg", "duration": "min", "duration": "max"}
)
station_duration_stats.show()

+--------------------+-----+
|       start_station|count|
+--------------------+-----+
|       2nd at Folsom| 7999|
|California Ave Ca...|  400|
|Powell at Post (U...| 6425|
| Golden Gate at Polk| 3646|
|Yerba Buena Cente...| 5523|
|   Market at Sansome|11431|
|         MLK Library| 1099|
|     Spear at Folsom| 5574|
|           Japantown|  885|
|Commercial at Mon...| 6014|
|Paseo de San Antonio|  856|
| San Salvador at 1st|  495|
|Rengstorff Avenue...|  501|
|     Townsend at 7th|13752|
|Civic Center BART...| 7760|
|         Ryland Park| 1120|
|San Jose Diridon ...| 4968|
|San Jose Civic Ce...|  774|
|          Mezes Park|  212|
|SJSU - San Salvad...|  494|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------+-----+
|       start_station|zip_code|count|
+--------------------+--------+-----+
|   Market at Sansome|   94544|   69|
|           Japantown|   95112|  443|
|   2nd at South Park|   94102|  125|
|Broadway St at Ba...|   94132|  246|
|Embarcadero

### Step 5: In SparkSQL it is also possible to create *User-Defined Functions (UDF)* to use within queries
1. See how this can be accomplished in [this page](https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/)
2. Show an example using the dataframes loaded in this notebook

In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Step 1: Define a UDF that will add 10 to the duration of the trips
def add_ten_to_duration(duration):
    return duration + 10 if duration is not None else None

# Register the UDF
add_ten_to_duration_udf = udf(add_ten_to_duration, IntegerType())

# Step 2: Use the UDF within a Spark SQL query or DataFrame transformation
trips_with_adjusted_duration = trips_df.withColumn(
    "adjusted_duration", add_ten_to_duration_udf(col("duration"))
)

# Step 3: Show the result
trips_with_adjusted_duration.select("start_station", "duration", "adjusted_duration").show(10)


NameError: name 'col' is not defined

### Step 6: Take two query examples from the previous steps and `explain` their execution plans

*Syntax:* `df.explain(True)`

In [7]:
# Example 1: Group by a single column and count
stations_count = trips_df.groupBy("start_station").count()

# Explain the execution plan for this query
stations_count.explain(True)

# Example 2: Group by a column and aggregate using sum
total_duration = trips_df.groupBy("start_station").sum("duration")

# Explain the execution plan for this query
total_duration.explain(True)

== Parsed Logical Plan ==
'Aggregate ['start_station], ['start_station, count(1) AS count#378L]
+- Repartition 9, true
   +- Relation [bike_id#67L,duration#68L,end_date#69,end_station#70,end_terminal#71L,start_date#72,start_station#73,start_terminal#74L,subscriber_type#75,trip_id#76L,zip_code#77L] json

== Analyzed Logical Plan ==
start_station: string, count: bigint
Aggregate [start_station#73], [start_station#73, count(1) AS count#378L]
+- Repartition 9, true
   +- Relation [bike_id#67L,duration#68L,end_date#69,end_station#70,end_terminal#71L,start_date#72,start_station#73,start_terminal#74L,subscriber_type#75,trip_id#76L,zip_code#77L] json

== Optimized Logical Plan ==
Aggregate [start_station#73], [start_station#73, count(1) AS count#378L]
+- Repartition 9, true
   +- Project [start_station#73]
      +- Relation [bike_id#67L,duration#68L,end_date#69,end_station#70,end_terminal#71L,start_date#72,start_station#73,start_terminal#74L,subscriber_type#75,trip_id#76L,zip_code#77L] json

=